In [14]:
import requests
import networkx as nx
import pandas as pd
import numpy as np
import geopy.distance
import matplotlib.pyplot as plt

In [15]:
def load_txt_file(file_name, columns, delimiter):
    data = pd.read_csv(file_name , delimiter=delimiter, names= columns)
    return data

In [16]:
columns = ['Airline', 'Airline ID', 'Source airport', 'Source airport ID', 'Destination airport'
           , 'Destination airport ID', 'Codeshare', 'Stops', 'Equipment']
airports_data = ['Airport ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude'
                 , 'Altitude', 'Timezone', 'DST', 'Tz Database time', 'Type', 'Source']
data = load_txt_file('routes.dat.txt', columns, delimiter=',')
lng_lot_data = load_txt_file('airports.dat.txt', airports_data, delimiter=',')

In [17]:
iata_hash = {}
# load all IATA code latitude and longitudes, IATA code as KEY
for index, row in lng_lot_data.iterrows():
    iata_hash[row["IATA"]] = {'Longitude':row['Longitude'], 'Latitude':row['Latitude']} 
def g_lg(key):
    hash_iata = iata_hash.get(key)
    return hash_iata['Latitude'], hash_iata['Longitude']

In [18]:
G = nx.DiGraph()

# create a list from source airport to destination airports
graph_data_iata = [i for i in zip(data['Source airport'].tolist(), data['Destination airport'].tolist())]
# compute distance from source airports to destincation airports
graph_data = [( g_data[0],g_data[1], geopy.distance.vincenty(g_lg(g_data[0]), g_lg(g_data[1])).km ) for g_data in graph_data_iata if (g_data[1] in iata_hash and g_data[0]) in iata_hash]

# add list of tumples with weight into the graph
G.add_weighted_edges_from(graph_data)

/home/muhammadumar/anaconda3/envs/ml_env/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Vincenty is deprecated and is going to be removed in geopy 2.0. Use `geopy.distance.geodesic` (or the default `geopy.distance.distance`) instead, which is more accurate and always converges.
  


In [30]:
src = 'KHI'
target = 'WUH'
# get all paths from source to destination with minimum edges = 2
simplest_path = [p for p in nx.all_simple_paths(G,source=src, target=target, cutoff=2)]

In [31]:
short_paths = dict()
for path in simplest_path:
    if len(path) > 2:
        short_paths[path[1]] = G.get_edge_data(path[0], path[1])['weight'] + G.get_edge_data(path[1], path[2])['weight']

min_val = max([short_paths.get(i) for i in short_paths])
key = ""
for i in short_paths:
    if short_paths.get(i) <= min_val:
        min_val = short_paths.get(i)
        key = i   

In [32]:
print(src , '->', key, '->', target)
print('Distance = ', min_val)

KHI -> CTU -> WUH
Distance =  4647.689253591717


In [33]:
lng_lot_data[lng_lot_data['IATA'] == 'CTU']

,Airport ID,Name,City,Country,IATA,ICAO,Latitude,Longitude,Altitude,Timezone,DST,Tz Database time,Type,Source
3199,3395,Chengdu Shuangliu International Airport,Chengdu,China,CTU,ZUUU,30.578501,103.946999,1625,8,U,Asia/Shanghai,airport,OurAirports
